# Salary estimator from listings

## Setup

In [24]:
%pip install pandas xgboost scikit-learn plotly # gensim

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


First we must import our packages to manage the dataset. Then we can import the data.

## Get the data

Now we can load the dataset.

In [25]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [26]:
repo_path = os.path.abspath('')
df = pd.read_csv(repo_path + '/archive/postings.csv')
df.head()

KeyboardInterrupt: 

In [ ]:
columns_to_drop = [
    'views','applies','original_listed_time','remote_allowed','job_posting_url','application_url','application_type',
    'expiry','closed_time','listed_time','posting_domain','sponsored','compensation_type','sponsored',
    ]
if 'views' in df.columns:
    df.drop(columns_to_drop, axis=1, inplace=True)
print(list(df.columns))

['job_id', 'company_name', 'title', 'description', 'max_salary', 'pay_period', 'location', 'company_id', 'med_salary', 'min_salary', 'formatted_work_type', 'formatted_experience_level', 'skills_desc', 'work_type', 'currency']


In [ ]:
pay_cols = ['max_salary','med_salary','min_salary']
df.dropna(thresh=1, subset=pay_cols, inplace=True)
len(df.index)

36073

In [50]:
graph_cols = pay_cols + ['company_name','title','location','pay_period']
pay_df = df[graph_cols].copy()
pay_df['state'] = pay_df['location'].str.split(',').str[1].str.strip().str.upper()
salary_df = pay_df.loc[pay_df['pay_period'] != 'HOURLY']
salary_state_df = salary_df[['state']+pay_cols].copy()
salary_state_df['avg_salary'] = salary_state_df[pay_cols].mean(axis=1)


def get_abnormal(arr: np.array):
    ser = pd.Series(arr.unique())
    result = ser[(ser != 'UNITED STATES') & (ser.str.len() != 2)]
    return result

print(get_abnormal(states))

3                                  NaN
21              OHIO METROPOLITAN AREA
31             TEXAS METROPOLITAN AREA
32                          CALIFORNIA
35    SOUTH CAROLINA METROPOLITAN AREA
38            OREGON METROPOLITAN AREA
44                        ALABAMA AREA
45            KANSAS METROPOLITAN AREA
46     MASSACHUSETTS METROPOLITAN AREA
56          NEBRASKA METROPOLITAN AREA
57                            NEW YORK
59                             MI AREA
60                            ILLINOIS
61         LOUISIANA METROPOLITAN AREA
62          NEW YORK METROPOLITAN AREA
66                              HAWAII
67                                OHIO
68                            DELAWARE
70    NORTH CAROLINA METROPOLITAN AREA
71                            COLORADO
72                            NEBRASKA
73                          WASHINGTON
74                       MASSACHUSETTS
75                               TEXAS
76                             AR AREA
77         WISCONSIN METR

In [51]:
state_map = [
    ['ALASKA', 'AK'],
    ['ALABAMA', 'AL'],
    ['ARKANSAS', 'AR'],
    ['ARIZONA', 'AZ'],
    ['CALIFORNIA', 'CA'],
    ['COLORADO', 'CO'],
    ['CONNECTICUT', 'CT'],
    ['DELAWARE', 'DE'],
    ['FLORIDA', 'FL'],
    ['GEORGIA', 'GA'],
    ['HAWAII', 'HI'],
    ['IOWA', 'IA'],
    ['IDAHO', 'ID'],
    ['ILLINOIS', 'IL'],
    ['INDIANA', 'IN'],
    ['KANSAS', 'KS'],
    ['KENTUCKY', 'KY'],
    ['LOUISIANA', 'LA'],
    ['MASSACHUSETTS', 'MA'],
    ['MARYLAND', 'MD'],
    ['MAINE', 'ME'],
    ['MICHIGAN', 'MI'],
    ['MINNESOTA', 'MN'],
    ['MISSOURI', 'MO'],
    ['MISSISSIPPI', 'MS'],
    ['MONTANA', 'MT'],
    ['NORTH CAROLINA', 'NC'],
    ['NORTH DAKOTA', 'ND'],
    ['NEBRASKA', 'NE'],
    ['NEW HAMPSHIRE', 'NH'],
    ['NEW JERSEY', 'NJ'],
    ['NEW MEXICO', 'NM'],
    ['NEVADA', 'NV'],
    ['NEW YORK', 'NY'],
    ['OHIO', 'OH'],
    ['OKLAHOMA', 'OK'],
    ['OREGON', 'OR'],
    ['PENNSYLVANIA', 'PA'],
    ['RHODE ISLAND', 'RI'],
    ['SOUTH CAROLINA', 'SC'],
    ['SOUTH DAKOTA', 'SD'],
    ['TENNESSEE', 'TN'],
    ['TEXAS', 'TX'],
    ['UTAH', 'UT'],
    ['VIRGINIA', 'VA'],
    ['VERMONT', 'VT'],
    ['WASHINGTON', 'WA'],
    ['WISCONSIN', 'WI'],
    ['WEST VIRGINIA', 'WV'],
    ['WYOMING', 'WY']
]
state_map_df = pd.DataFrame(state_map, columns=['name', 'abbr'])

def clean_state(state: str):
    if (state == state and len(state) != 2 and state != 'UNITED STATES'):
        state = state.lstrip(' AREA')
        state = state.lstrip(' METROPOLITAN')
        if state in state_map: 
            return state_map[state]
    return state

salary_state_df['state'].apply(clean_state)

print(get_abnormal(salary_state_df['state']))

#salary_state_df['state'] = np.where(, abbreviate(), arr)
print(salary_state_df['state'])

3                                  NaN
21              OHIO METROPOLITAN AREA
31             TEXAS METROPOLITAN AREA
32                          CALIFORNIA
35    SOUTH CAROLINA METROPOLITAN AREA
38            OREGON METROPOLITAN AREA
44                        ALABAMA AREA
45            KANSAS METROPOLITAN AREA
46     MASSACHUSETTS METROPOLITAN AREA
56          NEBRASKA METROPOLITAN AREA
57                            NEW YORK
59                             MI AREA
60                            ILLINOIS
61         LOUISIANA METROPOLITAN AREA
62          NEW YORK METROPOLITAN AREA
66                              HAWAII
67                                OHIO
68                            DELAWARE
70    NORTH CAROLINA METROPOLITAN AREA
71                            COLORADO
72                            NEBRASKA
73                          WASHINGTON
74                       MASSACHUSETTS
75                               TEXAS
76                             AR AREA
77         WISCONSIN METR

In [ ]:
fig_data = salary_state_df[['state', 'avg_salary']].groupby('state').median()
print(fig_data.head())
fig = px.bar(fig_data)
fig.show()

              avg_salary
state                   
AK              107000.0
AL               70000.0
ALABAMA AREA    115840.5
AR               65000.0
AR AREA         101243.5
